In [68]:
import pandas as pd

##Path to your list of target web domains (e.g., Tranco list: https://tranco-list.eu/)
file_path = ""

# Read the CSV file into a DataFrame   Note: (use header=None if using Tranco list)
df = pd.read_csv(file_path, header=None)

# Assuming format is Tranco list 
df.columns = ['Rank', 'Domain']


In [70]:
## Specific to creating dataset in our paper
top1k=df[df.Rank<=1000]
rest=df[df.Rank>1000]
sampled_rest_1k = rest.sample(n=1000, random_state=42)
dfeu=df[df['Domain'].str[-3:].isin(['.uk', '.eu','.be','.bg','.at','.hr','.cy','.cz', '.dk','.ee','.fi','.fr','.gr','.de','.hu', '.ie','.it','.lv','.lt','.lu','.mt','.nl', '.pl','.pt','.ro','.sk','.si','.es','.se'])]
top500eu=dfeu[:500]
rest_eu=dfeu[500:]
sampled_rest_eu = rest_eu.sample(n=500, random_state=42)
top1k['Type']='Top1k'
top500eu['Type']='TopEU'
sampled_rest_1k['Type']='Sampled1k'
sampled_rest_eu['Type']='SampledEU'
concatenated_df = pd.concat([top1k, top500eu,sampled_rest_1k,sampled_rest_eu], axis=0, ignore_index=True)
sorted_df = concatenated_df.sort_values(by='Rank', ascending=True).reset_index(drop=True)
final_df = sorted_df.drop_duplicates(subset=['Rank','Domain'], keep='first').reset_index(drop=True)
#final_df.to_csv('domain_list_ICWSM.csv')
final_df['EU']=False
for i in range(len(final_df)):
    if final_df.iloc[i,1][-3:] in ['.uk', '.eu','.be','.bg','.at','.hr','.cy','.cz', '.dk','.ee','.fi','.fr','.gr','.de','.hu', '.ie','.it','.lv','.lt','.lu','.mt','.nl', '.pl','.pt','.ro','.sk','.si','.es','.se']:
        final_df.iloc[i,-1]=True
        

In [124]:
## Check if you can access domain

final_df['DNS_Issue']=False
import socket
for i in range(len(final_df)):
    domain=final_df.iloc[i,1]
    try:
        ip_address = socket.gethostbyname(domain)
        #print(f"IP address of {domain}: {ip_address}")
    except socket.gaierror as e:
        print(f"Failed to resolve {domain}: {e}")
        final_df.iloc[i,-1]=True

Failed to resolve root-servers.net: [Errno 11001] getaddrinfo failed
Failed to resolve a-msedge.net: [Errno 11001] getaddrinfo failed
Failed to resolve akamai.net: [Errno 11001] getaddrinfo failed
Failed to resolve akamaiedge.net: [Errno 11001] getaddrinfo failed
Failed to resolve tiktokcdn.com: [Errno 11001] getaddrinfo failed
Failed to resolve akadns.net: [Errno 11001] getaddrinfo failed
Failed to resolve windowsupdate.com: [Errno 11001] getaddrinfo failed
Failed to resolve microsoftonline.com: [Errno 11001] getaddrinfo failed
Failed to resolve apple-dns.net: [Errno 11001] getaddrinfo failed
Failed to resolve trafficmanager.net: [Errno 11001] getaddrinfo failed
Failed to resolve office.net: [Errno 11001] getaddrinfo failed
Failed to resolve l-msedge.net: [Errno 11001] getaddrinfo failed
Failed to resolve gtld-servers.net: [Errno 11001] getaddrinfo failed
Failed to resolve t-msedge.net: [Errno 11001] getaddrinfo failed
Failed to resolve aaplimg.com: [Errno 11001] getaddrinfo failed
Fa

Failed to resolve pendingrenewaldeletion.com: [Errno 11001] getaddrinfo failed
Failed to resolve nel.goog: [Errno 11001] getaddrinfo failed
Failed to resolve cmediahub.ru: [Errno 11001] getaddrinfo failed
Failed to resolve name-services.com: [Errno 11001] getaddrinfo failed
Failed to resolve phicdn.net: [Errno 11001] getaddrinfo failed
Failed to resolve googlezip.net: [Errno 11001] getaddrinfo failed
Failed to resolve mozgcp.net: [Errno 11001] getaddrinfo failed
Failed to resolve akam.net: [Errno 11001] getaddrinfo failed
Failed to resolve grammarly.io: [Errno 11001] getaddrinfo failed
Failed to resolve dns-parking.com: [Errno 11001] getaddrinfo failed
Failed to resolve bdydns.com: [Errno 11001] getaddrinfo failed
Failed to resolve agkn.com: [Errno 11001] getaddrinfo failed
Failed to resolve licdn.com: [Errno 11001] getaddrinfo failed
Failed to resolve ovscdns.net: [Errno 11001] getaddrinfo failed
Failed to resolve myqcloud.com: [Errno 11004] getaddrinfo failed
Failed to resolve clouda

Failed to resolve iee.or.jp: [Errno 11001] getaddrinfo failed
Failed to resolve formas.toscana.it: [Errno 11001] getaddrinfo failed
Failed to resolve seaps.jp: [Errno 11001] getaddrinfo failed
Failed to resolve shopxxl.be: [Errno 11001] getaddrinfo failed
Failed to resolve cool-snack-shop.de: [Errno 11001] getaddrinfo failed
Failed to resolve team-sticker.de: [Errno 11001] getaddrinfo failed
Failed to resolve earswitchhealing.com: [Errno 11001] getaddrinfo failed
Failed to resolve anotdomain.net: [Errno 11001] getaddrinfo failed
Failed to resolve poznan-adwokat.pl: [Errno 11001] getaddrinfo failed
Failed to resolve dramago.info: [Errno 11001] getaddrinfo failed
Failed to resolve gtcfla.edu.cn: [Errno 11001] getaddrinfo failed
Failed to resolve sidekickopen10.com: [Errno 11001] getaddrinfo failed
Failed to resolve hosting2.dk: [Errno 11001] getaddrinfo failed
Failed to resolve coveris.academy: [Errno 11001] getaddrinfo failed
Failed to resolve wobline-ip.de: [Errno 11001] getaddrinfo fa

Failed to resolve gmtech.us: [Errno 11002] getaddrinfo failed


In [70]:
## List of domains to visit with Selenium crawler

domains=list(final_df[(final_df.DNS_Issue==False)]['Domain'])

In [87]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import WebDriverException, TimeoutException
import random
import time

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')
chrome_options.page_load_strategy = 'normal'
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
chrome_options.add_argument(f'user-agent={user_agent}')

#Can change this timeout
TIMEOUT = 60

In [88]:
# Function to Check IP Address and ensure it is in the location you want (assuming you use VPNs as we did)
def check_ip():
    driver = webdriver.Chrome(options=chrome_options)
    try:
        driver.get("https://httpbin.org/ip")
        ip_address = driver.find_element(By.TAG_NAME, "body").text
        print("Your IP address:", ip_address)
    finally:
        driver.quit()

In [89]:
# Function to Visit a URL and Take Screenshot
def visit_and_screenshot(domain, protocol="https"):
    url = f"{protocol}://{domain}"
    driver = webdriver.Chrome(options=chrome_options)
    try:
        driver.set_page_load_timeout(TIMEOUT)
        driver.get(url)
        actions = ActionChains(driver)

        # Simulate mouse movements
        for _ in range(3):
            x_offset = random.randint(1, 10)
            y_offset = random.randint(1, 30)
            actions.move_by_offset(x_offset, y_offset).perform()
            time.sleep(random.uniform(0.1, 0.6))

        # Save screenshot to wherever your image folder is located
        screenshot_path = f"[INSERT YOUR PATH]/{domain}.png"
        driver.save_screenshot(screenshot_path)
        print(f"Screenshot saved for {domain} at {screenshot_path}")
    finally:
        driver.quit()


In [90]:
# Process Domains - Can rerun on failed domain set with longer timeout
failed_domains = []
count = 0
for domain in domains:
    count += 1
    print(f"Processing {domain} ({count}/{len(domains)})")

    # Check IP every 50 domains
    if count % 50 == 0:
        check_ip()

    # Attempt HTTPS with www
    try:
        visit_and_screenshot(f"www.{domain}", protocol="https")
    except TimeoutException:
        print(f"Timeout occurred for {domain}")
        failed_domains.append(domain)
    except WebDriverException as e:
        print(f"HTTPS www failed for {domain}: {e}")

        # Retry with HTTP without www
        try:
            visit_and_screenshot(domain, protocol="http")
        except TimeoutException:
            print(f"Timeout occurred for {domain}")
            failed_domains.append(domain)
        except WebDriverException as e2:
            print(f"HTTP failed for {domain}: {e2}")

# Log Failed Domains
print("Failed domains:", failed_domains)


Processing msedge.net (1/2579)
HTTPS www failed for msedge.net: Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: chrome=131.0.6778.109)
Stacktrace:
	GetHandleVerifier [0x00007FF7A989FB05+28789]
	(No symbol) [0x00007FF7A98086E0]
	(No symbol) [0x00007FF7A96A592A]
	(No symbol) [0x00007FF7A96A3028]
	(No symbol) [0x00007FF7A9693BF9]
	(No symbol) [0x00007FF7A969594F]
	(No symbol) [0x00007FF7A9693EBF]
	(No symbol) [0x00007FF7A969371B]
	(No symbol) [0x00007FF7A969365A]
	(No symbol) [0x00007FF7A9691274]
	(No symbol) [0x00007FF7A9691B3C]
	(No symbol) [0x00007FF7A96A89FA]
	(No symbol) [0x00007FF7A974045E]
	(No symbol) [0x00007FF7A971F43A]
	(No symbol) [0x00007FF7A973F654]
	(No symbol) [0x00007FF7A971F1E3]
	(No symbol) [0x00007FF7A96EA938]
	(No symbol) [0x00007FF7A96EBAA1]
	GetHandleVerifier [0x00007FF7A9BD933D+3410093]
	GetHandleVerifier [0x00007FF7A9BEE7DD+3497293]
	GetHandleVerifier [0x00007FF7A9BE2A73+3448803]
	GetHandleVerifier [0x00007FF7A9967BBB+848171]
	(No symbol) [0

HTTP failed for jsdelivr.net: Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: chrome=131.0.6778.109)
Stacktrace:
	GetHandleVerifier [0x00007FF7A989FB05+28789]
	(No symbol) [0x00007FF7A98086E0]
	(No symbol) [0x00007FF7A96A592A]
	(No symbol) [0x00007FF7A96A3028]
	(No symbol) [0x00007FF7A9693BF9]
	(No symbol) [0x00007FF7A969594F]
	(No symbol) [0x00007FF7A9693EBF]
	(No symbol) [0x00007FF7A969371B]
	(No symbol) [0x00007FF7A969365A]
	(No symbol) [0x00007FF7A9691274]
	(No symbol) [0x00007FF7A9691B3C]
	(No symbol) [0x00007FF7A96A89FA]
	(No symbol) [0x00007FF7A974045E]
	(No symbol) [0x00007FF7A971F43A]
	(No symbol) [0x00007FF7A973F654]
	(No symbol) [0x00007FF7A971F1E3]
	(No symbol) [0x00007FF7A96EA938]
	(No symbol) [0x00007FF7A96EBAA1]
	GetHandleVerifier [0x00007FF7A9BD933D+3410093]
	GetHandleVerifier [0x00007FF7A9BEE7DD+3497293]
	GetHandleVerifier [0x00007FF7A9BE2A73+3448803]
	GetHandleVerifier [0x00007FF7A9967BBB+848171]
	(No symbol) [0x00007FF7A9813C3F]
	(No symbol) [0

HTTP failed for imcmdb.net: Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: chrome=131.0.6778.109)
Stacktrace:
	GetHandleVerifier [0x00007FF7A989FB05+28789]
	(No symbol) [0x00007FF7A98086E0]
	(No symbol) [0x00007FF7A96A592A]
	(No symbol) [0x00007FF7A96A3028]
	(No symbol) [0x00007FF7A9693BF9]
	(No symbol) [0x00007FF7A969594F]
	(No symbol) [0x00007FF7A9693EBF]
	(No symbol) [0x00007FF7A969371B]
	(No symbol) [0x00007FF7A969365A]
	(No symbol) [0x00007FF7A9691274]
	(No symbol) [0x00007FF7A9691B3C]
	(No symbol) [0x00007FF7A96A89FA]
	(No symbol) [0x00007FF7A974045E]
	(No symbol) [0x00007FF7A971F43A]
	(No symbol) [0x00007FF7A973F654]
	(No symbol) [0x00007FF7A971F1E3]
	(No symbol) [0x00007FF7A96EA938]
	(No symbol) [0x00007FF7A96EBAA1]
	GetHandleVerifier [0x00007FF7A9BD933D+3410093]
	GetHandleVerifier [0x00007FF7A9BEE7DD+3497293]
	GetHandleVerifier [0x00007FF7A9BE2A73+3448803]
	GetHandleVerifier [0x00007FF7A9967BBB+848171]
	(No symbol) [0x00007FF7A9813C3F]
	(No symbol) [0x0

Timeout occurred for ipv4only.arpa
Processing spaceweb.pro (28/2579)
HTTPS www failed for spaceweb.pro: Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: chrome=131.0.6778.109)
Stacktrace:
	GetHandleVerifier [0x00007FF7A989FB05+28789]
	(No symbol) [0x00007FF7A98086E0]
	(No symbol) [0x00007FF7A96A592A]
	(No symbol) [0x00007FF7A96A3028]
	(No symbol) [0x00007FF7A9693BF9]
	(No symbol) [0x00007FF7A969594F]
	(No symbol) [0x00007FF7A9693EBF]
	(No symbol) [0x00007FF7A969371B]
	(No symbol) [0x00007FF7A969365A]
	(No symbol) [0x00007FF7A9691274]
	(No symbol) [0x00007FF7A9691B3C]
	(No symbol) [0x00007FF7A96A89FA]
	(No symbol) [0x00007FF7A974045E]
	(No symbol) [0x00007FF7A971F43A]
	(No symbol) [0x00007FF7A973F654]
	(No symbol) [0x00007FF7A971F1E3]
	(No symbol) [0x00007FF7A96EA938]
	(No symbol) [0x00007FF7A96EBAA1]
	GetHandleVerifier [0x00007FF7A9BD933D+3410093]
	GetHandleVerifier [0x00007FF7A9BEE7DD+3497293]
	GetHandleVerifier [0x00007FF7A9BE2A73+3448803]
	GetHandleVerifier [0x

HTTP failed for typekit.net: Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: chrome=131.0.6778.109)
Stacktrace:
	GetHandleVerifier [0x00007FF7A989FB05+28789]
	(No symbol) [0x00007FF7A98086E0]
	(No symbol) [0x00007FF7A96A592A]
	(No symbol) [0x00007FF7A96A3028]
	(No symbol) [0x00007FF7A9693BF9]
	(No symbol) [0x00007FF7A969594F]
	(No symbol) [0x00007FF7A9693EBF]
	(No symbol) [0x00007FF7A969371B]
	(No symbol) [0x00007FF7A969365A]
	(No symbol) [0x00007FF7A9691274]
	(No symbol) [0x00007FF7A9691B3C]
	(No symbol) [0x00007FF7A96A89FA]
	(No symbol) [0x00007FF7A974045E]
	(No symbol) [0x00007FF7A971F43A]
	(No symbol) [0x00007FF7A973F654]
	(No symbol) [0x00007FF7A971F1E3]
	(No symbol) [0x00007FF7A96EA938]
	(No symbol) [0x00007FF7A96EBAA1]
	GetHandleVerifier [0x00007FF7A9BD933D+3410093]
	GetHandleVerifier [0x00007FF7A9BEE7DD+3497293]
	GetHandleVerifier [0x00007FF7A9BE2A73+3448803]
	GetHandleVerifier [0x00007FF7A9967BBB+848171]
	(No symbol) [0x00007FF7A9813C3F]
	(No symbol) [0x

...

HTTP failed for powerade-service.fr: Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: chrome=131.0.6778.109)
Stacktrace:
	GetHandleVerifier [0x00007FF7A989FB05+28789]
	(No symbol) [0x00007FF7A98086E0]
	(No symbol) [0x00007FF7A96A592A]
	(No symbol) [0x00007FF7A96A3028]
	(No symbol) [0x00007FF7A9693BF9]
	(No symbol) [0x00007FF7A969594F]
	(No symbol) [0x00007FF7A9693EBF]
	(No symbol) [0x00007FF7A969371B]
	(No symbol) [0x00007FF7A969365A]
	(No symbol) [0x00007FF7A9691274]
	(No symbol) [0x00007FF7A9691B3C]
	(No symbol) [0x00007FF7A96A89FA]
	(No symbol) [0x00007FF7A974045E]
	(No symbol) [0x00007FF7A971F43A]
	(No symbol) [0x00007FF7A973F654]
	(No symbol) [0x00007FF7A971F1E3]
	(No symbol) [0x00007FF7A96EA938]
	(No symbol) [0x00007FF7A96EBAA1]
	GetHandleVerifier [0x00007FF7A9BD933D+3410093]
	GetHandleVerifier [0x00007FF7A9BEE7DD+3497293]
	GetHandleVerifier [0x00007FF7A9BE2A73+3448803]
	GetHandleVerifier [0x00007FF7A9967BBB+848171]
	(No symbol) [0x00007FF7A9813C3F]
	(No sym

Processing aluminijumskeograde.com (124/2579)
HTTPS www failed for aluminijumskeograde.com: Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: chrome=131.0.6778.109)
Stacktrace:
	GetHandleVerifier [0x00007FF7A989FB05+28789]
	(No symbol) [0x00007FF7A98086E0]
	(No symbol) [0x00007FF7A96A592A]
	(No symbol) [0x00007FF7A96A3028]
	(No symbol) [0x00007FF7A9693BF9]
	(No symbol) [0x00007FF7A969594F]
	(No symbol) [0x00007FF7A9693EBF]
	(No symbol) [0x00007FF7A969371B]
	(No symbol) [0x00007FF7A969365A]
	(No symbol) [0x00007FF7A9691274]
	(No symbol) [0x00007FF7A9691B3C]
	(No symbol) [0x00007FF7A96A89FA]
	(No symbol) [0x00007FF7A974045E]
	(No symbol) [0x00007FF7A971F43A]
	(No symbol) [0x00007FF7A973F654]
	(No symbol) [0x00007FF7A971F1E3]
	(No symbol) [0x00007FF7A96EA938]
	(No symbol) [0x00007FF7A96EBAA1]
	GetHandleVerifier [0x00007FF7A9BD933D+3410093]
	GetHandleVerifier [0x00007FF7A9BEE7DD+3497293]
	GetHandleVerifier [0x00007FF7A9BE2A73+3448803]
	GetHandleVerifier [0x00007FF7A996

Screenshot saved for reckitt.us at C:/Users/riley/CookieBanners_ICWSM_NY/reckitt.us.png
Processing r3capitalpartner.jp (128/2579)
HTTPS www failed for r3capitalpartner.jp: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=131.0.6778.109)
Stacktrace:
	GetHandleVerifier [0x00007FF7A989FB05+28789]
	(No symbol) [0x00007FF7A98086E0]
	(No symbol) [0x00007FF7A96A592A]
	(No symbol) [0x00007FF7A96A3028]
	(No symbol) [0x00007FF7A9693BF9]
	(No symbol) [0x00007FF7A969594F]
	(No symbol) [0x00007FF7A9693EBF]
	(No symbol) [0x00007FF7A969371B]
	(No symbol) [0x00007FF7A969365A]
	(No symbol) [0x00007FF7A9691274]
	(No symbol) [0x00007FF7A9691B3C]
	(No symbol) [0x00007FF7A96A89FA]
	(No symbol) [0x00007FF7A974045E]
	(No symbol) [0x00007FF7A971F43A]
	(No symbol) [0x00007FF7A973F654]
	(No symbol) [0x00007FF7A971F1E3]
	(No symbol) [0x00007FF7A96EA938]
	(No symbol) [0x00007FF7A96EBAA1]
	GetHandleVerifier [0x00007FF7A9BD933D+3410093]
	GetHandleVerifier [0x00007FF7A9BEE7DD+3497293]
	GetH

Processing affiliate.co.uk (132/2579)
HTTPS www failed for affiliate.co.uk: Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: chrome=131.0.6778.109)
Stacktrace:
	GetHandleVerifier [0x00007FF7A989FB05+28789]
	(No symbol) [0x00007FF7A98086E0]
	(No symbol) [0x00007FF7A96A592A]
	(No symbol) [0x00007FF7A96A3028]
	(No symbol) [0x00007FF7A9693BF9]
	(No symbol) [0x00007FF7A969594F]
	(No symbol) [0x00007FF7A9693EBF]
	(No symbol) [0x00007FF7A969371B]
	(No symbol) [0x00007FF7A969365A]
	(No symbol) [0x00007FF7A9691274]
	(No symbol) [0x00007FF7A9691B3C]
	(No symbol) [0x00007FF7A96A89FA]
	(No symbol) [0x00007FF7A974045E]
	(No symbol) [0x00007FF7A971F43A]
	(No symbol) [0x00007FF7A973F654]
	(No symbol) [0x00007FF7A971F1E3]
	(No symbol) [0x00007FF7A96EA938]
	(No symbol) [0x00007FF7A96EBAA1]
	GetHandleVerifier [0x00007FF7A9BD933D+3410093]
	GetHandleVerifier [0x00007FF7A9BEE7DD+3497293]
	GetHandleVerifier [0x00007FF7A9BE2A73+3448803]
	GetHandleVerifier [0x00007FF7A9967BBB+848171]
	(N

HTTP failed for bluelavazza.com: Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: chrome=131.0.6778.109)
Stacktrace:
	GetHandleVerifier [0x00007FF7A989FB05+28789]
	(No symbol) [0x00007FF7A98086E0]
	(No symbol) [0x00007FF7A96A592A]
	(No symbol) [0x00007FF7A96A3028]
	(No symbol) [0x00007FF7A9693BF9]
	(No symbol) [0x00007FF7A969594F]
	(No symbol) [0x00007FF7A9693EBF]
	(No symbol) [0x00007FF7A969371B]
	(No symbol) [0x00007FF7A969365A]
	(No symbol) [0x00007FF7A9691274]
	(No symbol) [0x00007FF7A9691B3C]
	(No symbol) [0x00007FF7A96A89FA]
	(No symbol) [0x00007FF7A974045E]
	(No symbol) [0x00007FF7A971F43A]
	(No symbol) [0x00007FF7A973F654]
	(No symbol) [0x00007FF7A971F1E3]
	(No symbol) [0x00007FF7A96EA938]
	(No symbol) [0x00007FF7A96EBAA1]
	GetHandleVerifier [0x00007FF7A9BD933D+3410093]
	GetHandleVerifier [0x00007FF7A9BEE7DD+3497293]
	GetHandleVerifier [0x00007FF7A9BE2A73+3448803]
	GetHandleVerifier [0x00007FF7A9967BBB+848171]
	(No symbol) [0x00007FF7A9813C3F]
	(No symbol)